<a href="https://colab.research.google.com/github/sanghyun412/jolapp-auto/blob/main/15%EB%B6%84%EB%B4%89_%EC%8B%A4%EC%A0%84_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyupbit
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from datetime import datetime, timedelta
import pytz

def get_market_list():
    markets = pyupbit.get_tickers(fiat="KRW")
    return markets

def fetch_ohlcv(market, interval="minute15", count=200):
    try:
        df = pyupbit.get_ohlcv(market, interval=interval, count=count)
        if df is None or df.empty:
            return None
        return df
    except Exception as e:
        print(f"[Error] {market} 데이터 수집 실패: {e}")
        return None

def get_market_condition():
    df_day = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=30)
    rsi = RSIIndicator(df_day['close'], window=14).rsi().iloc[-1]
    if rsi > 60:
        return 'bull'  # 상승장
    elif rsi < 40:
        return 'bear'  # 하락장
    else:
        return 'side'  # 횡보장

def calc_profit_target(market_condition):
    if market_condition == 'bull':
        return 0.07
    elif market_condition == 'bear':
        return 0.04
    else:
        return 0.055

def get_grade(win_rate):
    if win_rate >= 65:
        return "🟩 강력추천"
    elif win_rate >= 30:
        return "🟦 일반추천"
    else:
        return "⛔ 비추천"

def backtest(market, df, profit_target):
    entry_price = df['close'].iloc[-2]
    tp = entry_price * (1 + profit_target)
    sl = entry_price * (1 - profit_target / 2)
    rr = (tp - entry_price) / (entry_price - sl)

    success_trades = 0
    total_trades = 0
    tp_times = []
    sl_times = []

    for i in range(len(df) - 1):
        open_time = df.index[i]
        entry = df['close'].iloc[i]

        tp_price = entry * (1 + profit_target)
        sl_price = entry * (1 - profit_target / 2)

        tp_reached = False
        sl_reached = False
        time_to_tp = None
        time_to_sl = None

        for j in range(i + 1, len(df)):
            high = df['high'].iloc[j]
            low = df['low'].iloc[j]
            cur_time = df.index[j]

            if not tp_reached and high >= tp_price:
                tp_reached = True
                time_to_tp = (cur_time - open_time).total_seconds() / 3600
            if not sl_reached and low <= sl_price:
                sl_reached = True
                time_to_sl = (cur_time - open_time).total_seconds() / 3600
            if tp_reached or sl_reached:
                break

        if tp_reached and (not sl_reached or time_to_tp <= time_to_sl):
            success_trades += 1
            tp_times.append(time_to_tp if time_to_tp is not None else np.nan)
            sl_times.append(time_to_sl if time_to_sl is not None else np.nan)
        elif sl_reached:
            tp_times.append(time_to_tp if time_to_tp is not None else np.nan)
            sl_times.append(time_to_sl if time_to_sl is not None else np.nan)

        total_trades += 1

    win_rate = (success_trades / total_trades) * 100 if total_trades > 0 else 0
    avg_tp_time = np.nanmean(tp_times) if len(tp_times) > 0 else np.nan
    avg_sl_time = np.nanmean(sl_times) if len(sl_times) > 0 else np.nan

    return {
        "market": market,
        "entry_price": entry_price,
        "tp": tp,
        "sl": sl,
        "rr": rr,
        "success_trades": success_trades,
        "total_trades": total_trades,
        "win_rate": win_rate,
        "avg_tp_time": avg_tp_time,
        "avg_sl_time": avg_sl_time,
    }

def format_num(num):
    if num >= 1000:
        return f"{num:,.0f}"
    elif num >= 1:
        return f"{num:,.3f}"
    else:
        return f"{num:.6f}"

def main():
    markets = get_market_list()
    market_condition = get_market_condition()
    profit_target = calc_profit_target(market_condition)
    print(f"시장 상황: {market_condition}, 목표 수익률: {profit_target*100:.1f}%")

    results = []
    for market in markets:
        df = fetch_ohlcv(market, interval="minute15", count=200)
        if df is None or len(df) < 100:
            print(f"[Skip] {market} - 데이터 부족 또는 오류")
            continue
        res = backtest(market, df, profit_target)
        results.append(res)

    results = sorted(results, key=lambda x: x['win_rate'], reverse=True)
    top_results = results[:15]

    now_kst = datetime.now(pytz.timezone('Asia/Seoul')).strftime("%m/%d %H:%M (KST)")

    msg = f"📊 분석 시각: {now_kst}\n"
    msg += f"📈 시장상태: {market_condition}\n\n"
    msg += f"✅ [15분봉 단타 추천] {len(top_results)}개\n\n"

    for c in top_results:
        tp_time_str = f"{c['avg_tp_time']:.1f}시간" if not np.isnan(c['avg_tp_time']) else 'N/A'
        sl_time_str = f"{c['avg_sl_time']:.1f}시간" if not np.isnan(c['avg_sl_time']) else 'N/A'

        grade = get_grade(c['win_rate'])

        msg += (
            f"⚡ {c['market'].split('-')[1]} ({grade})\n"
            f"- 진입가: {format_num(c['entry_price'])} / 익절: {format_num(c['tp'])} / 손절: {format_num(c['sl'])}\n"
            f"- 예상 수익률: +{profit_target*100:.1f}% / R:R: {c['rr']:.2f}\n"
            f"- 승률: {c['success_trades']}건 / {c['total_trades']}건 → 승률 {c['win_rate']:.1f}%\n"
            f"- 익절 도달시간: {tp_time_str} / 손절 도달시간: {sl_time_str}\n\n"
        )

    print(msg)

if __name__ == "__main__":
    main()


시장 상황: side, 목표 수익률: 5.5%


<ipython-input-46-840547049>:96: RuntimeWarning: Mean of empty slice
  avg_tp_time = np.nanmean(tp_times) if len(tp_times) > 0 else np.nan
